In [1]:
import cv2
import numpy as np
import os
import glob
import json
import shutil
from pathlib import Path
from tqdm import tqdm

def find_project_root(marker=".gitignore"):
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent.resolve()
    raise FileNotFoundError(
        f"Project root marker '{marker}' not found starting from {current}")

In [2]:
root = find_project_root()

intrinsic_path = os.path.join(root, "output", "intrinsic_params.json")
with open(intrinsic_path, "r") as f:
    intrinsics = json.load(f)

extrinsic_path = os.path.join(root, "output", "keep_best_stereo_params.json")
with open(extrinsic_path, "r") as f:
    stereo_extrinsics = json.load(f)
    
full_stereo_params_path = os.path.join(root, "output", "full_stereo_params.json")
with open(full_stereo_params_path, "r") as f:
    full_stereo_params = json.load(f)

## Input
- 2D ball points from left and right CAM A, same frame
- K and D of left and right A
- full stereo transform to align the stereos

## Pipeline
- Undistort 2D points (from both cameras)
- Build projection matrices
- Triangulate 3D point
- Reproject back to each camera to check error

